## __This is first basic tutorial for `tf.keras`.__

#### We use MNIST handwrite dataset.
#### And construct simple fully-connected(FC) layers.

### __1. Load dependencies__

First, we can load `tf.keras` and other python libraries.

In [21]:
import numpy as np
np.random.seed(2 ** 10)

import tensorflow as tf
print("[INFO] Tensorflow version is {}".format(tf.__version__))  # 1.11.0

[INFO] Tensorflow version is 1.11.0


And then, we can load tensorflow eager model for better tensor management.

In [10]:
tf.enable_eager_execution()

### 2. Load dataset and pre-processing

In [26]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

There are total 3 steps in data pre-processing.

* 1) Convert data type.
* 2) Reshape data.
* 3) Normalize data.

We only perform the division of the dataset by 255 because dataset is 1 channel images and the distribution of data is simple.

In [27]:
x_train = x_train.astype(np.float32)     # convert data type
x_test = x_test.astype(np.float32)
x_train = np.reshape(x_train, (len(x_train), 784))  # reshape: (28, 28) --> (784) (i.e., flatten 2D matrix to 1D vector)
x_test = np.reshape(x_test, (len(x_test), 784))
x_train /= 255.  # normalization
x_test /= 255.

We use `tf.keras.utils` for one-hot encoding.

In [28]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10).astype(np.float32)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10).astype(np.float32)

print("Shape of x_train is {}".format(np.shape(x_train)))
print("Shape of x_test is {}".format(np.shape(x_test)))
print("Shape of y_train is {}".format(np.shape(y_train)))
print("Shape of y_test is {}".format(np.shape(y_test)))

Shape of x_train is (60000, 784)
Shape of x_test is (10000, 784)
Shape of y_train is (60000, 10)
Shape of y_test is (10000, 10)


### 3. Construct tf.keras model

We can construct tensorflow model by using `tf.keras.layers`.

It's a lot easier than using an existing tensorflow, and the management of the tensor is clear.

In [29]:
# Build a tf.keras model
inputs = tf.keras.Input(shape=(784,))

fc_1 = tf.keras.layers.Dense(512, activation=tf.nn.relu)(inputs)
fc_2 = tf.keras.layers.Dense(256, activation=tf.nn.relu)(fc_1)
probabilities = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(fc_2)

#probabilities = tf.keras.layers.Dense(10, )(logits)

model = tf.keras.Model(inputs=[inputs], outputs=[probabilities])
model.summary()
print("[INFO] Total parameters of model is {}".format(model.count_params()))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
[INFO] Total parameters of model is 535818


Now, we have to specify model's configuration.

In [30]:
opt = tf.train.RMSPropOptimizer(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt, metrics=['accuracy'])

### 4. Dataset pipeline

We use the `tf.data` tensor flow library to construct the link between dataset and model.

Of course you can use Keras's basic `model.fit` method, but this example uses `tf.data`.

In [31]:
BATCH_SIZE = 64
SHUFFLE_SIZE = 10000 

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(SHUFFLE_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

print(dataset)

<BatchDataset shapes: ((?, 784), (?, 10)), types: (tf.float32, tf.float32)>


You should note the batch dataset shape.

Where shape must match the shape in `tf.summary` above.

In this situation,? indicates __64__, the batch size we set.

### 5. Train model

Finally, train our model for checking accuracy!!

In [32]:
for epoch in range(10):
    for (train_images, train_labels), (test_images, test_labels) in zip(dataset, test_dataset):
        loss, acc = model.train_on_batch(train_images, train_labels)
        test_loss, test_acc = model.test_on_batch(test_images, test_labels)
    print('Epoch #{}\t Loss: #{}\tAccuracy: #{}'.format(epoch + 1, test_loss, test_acc))

test_loss, test_acc = model.evaluate(x_test, y_test)
print('\nTest Model \t\t Loss: %.6f\tAccuracy: %.6f' % (test_loss, test_acc))

Epoch #1	 Loss: #0.9341843724250793	Accuracy: #0.75
Epoch #2	 Loss: #0.15050853788852692	Accuracy: #0.9375
Epoch #3	 Loss: #0.09951159358024597	Accuracy: #0.9375
Epoch #4	 Loss: #0.13128797709941864	Accuracy: #0.9375
Epoch #5	 Loss: #0.05193028226494789	Accuracy: #1.0
Epoch #6	 Loss: #0.12501972913742065	Accuracy: #0.9375
Epoch #7	 Loss: #0.0003079454763792455	Accuracy: #1.0
Epoch #8	 Loss: #0.7894665598869324	Accuracy: #0.875
Epoch #9	 Loss: #0.056634288281202316	Accuracy: #0.9375
Epoch #10	 Loss: #0.0029337259475141764	Accuracy: #1.0
313/313 [==============================] - 1s 2ms/step

Test Model 		 Loss: 0.150603	Accuracy: 0.963300


It's not bad :)

You can change `tf.keras` models and dataset configuration yourself.

This concludes the **first** example. 

Next, let's design a tensorflow model that includes convolution using a __CIFAR__ dataset.